In [1]:
!pip install pyspark

In [2]:
import numpy as np 
import pandas as pd 
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataStorm').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/25 16:07:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:
df_pyspark = spark.read.option('header','true').csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
df_pyspark.show()

+--------------------+--------------------+----------+-----------+-------+----------+-------------+
|    item_description|    transaction_date|invoice_id|customer_id|shop_id|item_price|quantity_sold|
+--------------------+--------------------+----------+-----------+-------+----------+-------------+
|  ORANGE BARLEY 1.5L|2021-12-11T00:00:...|       147|       BGXA|SHOP008|       220|            2|
|    GINGER BEER 1.5L|2021-10-17T00:00:...|       371|       IA25|SHOP112|       220|            2|
|     TONIC PET 500ML|2021-12-13T00:00:...|       484|       VN7V|SHOP008|       160|            2|
|       CREAM SODA 1L|2021-12-13T00:00:...|       484|       VN7V|SHOP008|       150|            2|
|STRAWBERRY MILK 1...|2021-10-23T00:00:...|      1310|       7S00|SHOP112|       210|            5|
|       CREAM SODA 1L|2021-10-30T00:00:...|      1624|       KZB3|SHOP055|       150|            2|
|     CREAM SODA 1.5L|2021-10-28T00:00:...|      2024|       0PDI|SHOP112|       220|            2|


In [5]:
df_pyspark = df_pyspark.withColumn('Sale',df_pyspark['item_price']*df_pyspark['quantity_sold'])

In [6]:
from pyspark.sql.functions import col,isnan, when, count

check_columns = df_pyspark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_pyspark.columns])
check_columns.show()

+----------------+----------------+----------+-----------+-------+----------+-------------+----+
|item_description|transaction_date|invoice_id|customer_id|shop_id|item_price|quantity_sold|Sale|
+----------------+----------------+----------+-----------+-------+----------+-------------+----+
|               0|               0|         0|          0|      0|         0|            0|   0|
+----------------+----------------+----------+-----------+-------+----------+-------------+----+



**there is not any null value in the dataframe**

In [7]:
df_pyspark.drop("item_price").show()

+--------------------+--------------------+----------+-----------+-------+-------------+------+
|    item_description|    transaction_date|invoice_id|customer_id|shop_id|quantity_sold|  Sale|
+--------------------+--------------------+----------+-----------+-------+-------------+------+
|  ORANGE BARLEY 1.5L|2021-12-11T00:00:...|       147|       BGXA|SHOP008|            2| 440.0|
|    GINGER BEER 1.5L|2021-10-17T00:00:...|       371|       IA25|SHOP112|            2| 440.0|
|     TONIC PET 500ML|2021-12-13T00:00:...|       484|       VN7V|SHOP008|            2| 320.0|
|       CREAM SODA 1L|2021-12-13T00:00:...|       484|       VN7V|SHOP008|            2| 300.0|
|STRAWBERRY MILK 1...|2021-10-23T00:00:...|      1310|       7S00|SHOP112|            5|1050.0|
|       CREAM SODA 1L|2021-10-30T00:00:...|      1624|       KZB3|SHOP055|            2| 300.0|
|     CREAM SODA 1.5L|2021-10-28T00:00:...|      2024|       0PDI|SHOP112|            2| 440.0|
|    TWISTEE APPLE 1L|2021-10-31T00:00:.

In [8]:
from pyspark.sql.functions import countDistinct
from pyspark.sql import Window

New_alias = df_pyspark.groupBy("transaction_date").agg(countDistinct("invoice_id").alias("Sale_in_each_day"))

In [9]:
df_pyspark = df_pyspark.join(New_alias, "transaction_date", "left")
df_pyspark.show()

+--------------------+--------------------+----------+-----------+-------+----------+-------------+------+----------------+
|    transaction_date|    item_description|invoice_id|customer_id|shop_id|item_price|quantity_sold|  Sale|Sale_in_each_day|
+--------------------+--------------------+----------+-----------+-------+----------+-------------+------+----------------+
|2021-12-11T00:00:...|  ORANGE BARLEY 1.5L|       147|       BGXA|SHOP008|       220|            2| 440.0|           11713|
|2021-10-17T00:00:...|    GINGER BEER 1.5L|       371|       IA25|SHOP112|       220|            2| 440.0|            7440|
|2021-12-13T00:00:...|     TONIC PET 500ML|       484|       VN7V|SHOP008|       160|            2| 320.0|            5976|
|2021-12-13T00:00:...|       CREAM SODA 1L|       484|       VN7V|SHOP008|       150|            2| 300.0|            5976|
|2021-10-23T00:00:...|STRAWBERRY MILK 1...|      1310|       7S00|SHOP112|       210|            5|1050.0|            7997|
|2021-10

In [10]:
New_alias_1 = df_pyspark.groupBy("shop_id").agg(countDistinct("item_description").alias("Distinct_item_count_sold"))
df_pyspark = df_pyspark.join(New_alias_1, "shop_id", "left")
df_pyspark.show()

+-------+--------------------+--------------------+----------+-----------+----------+-------------+------+----------------+------------------------+
|shop_id|    transaction_date|    item_description|invoice_id|customer_id|item_price|quantity_sold|  Sale|Sale_in_each_day|Distinct_item_count_sold|
+-------+--------------------+--------------------+----------+-----------+----------+-------------+------+----------------+------------------------+
|SHOP008|2021-12-11T00:00:...|  ORANGE BARLEY 1.5L|       147|       BGXA|       220|            2| 440.0|           11713|                      32|
|SHOP112|2021-10-17T00:00:...|    GINGER BEER 1.5L|       371|       IA25|       220|            2| 440.0|            7440|                      36|
|SHOP008|2021-12-13T00:00:...|     TONIC PET 500ML|       484|       VN7V|       160|            2| 320.0|            5976|                      32|
|SHOP008|2021-12-13T00:00:...|       CREAM SODA 1L|       484|       VN7V|       150|            2| 300.0|

In [11]:
#Check number of Customors Engaged
New_alias_2 = df_pyspark.groupBy("shop_id").agg(countDistinct("customer_id").alias("Customors_engaged"))
df_pyspark = df_pyspark.join(New_alias_2, "shop_id", "left")
df_pyspark.show()

+-------+--------------------+--------------------+----------+-----------+----------+-------------+------+----------------+------------------------+-----------------+
|shop_id|    transaction_date|    item_description|invoice_id|customer_id|item_price|quantity_sold|  Sale|Sale_in_each_day|Distinct_item_count_sold|Customors_engaged|
+-------+--------------------+--------------------+----------+-----------+----------+-------------+------+----------------+------------------------+-----------------+
|SHOP008|2021-12-11T00:00:...|  ORANGE BARLEY 1.5L|       147|       BGXA|       220|            2| 440.0|           11713|                      32|              189|
|SHOP112|2021-10-17T00:00:...|    GINGER BEER 1.5L|       371|       IA25|       220|            2| 440.0|            7440|                      36|             1308|
|SHOP008|2021-12-13T00:00:...|     TONIC PET 500ML|       484|       VN7V|       160|            2| 320.0|            5976|                      32|              189

In [12]:
df_pyspark_1 = df_pyspark.drop('transaction_date','item_description','item_price','customer_id','invoice_id')

In [13]:
df_pyspark_1.show()

+-------+-------------+------+----------------+------------------------+-----------------+
|shop_id|quantity_sold|  Sale|Sale_in_each_day|Distinct_item_count_sold|Customors_engaged|
+-------+-------------+------+----------------+------------------------+-----------------+
|SHOP008|            2| 440.0|           11713|                      32|              189|
|SHOP112|            2| 440.0|            7440|                      36|             1308|
|SHOP008|            2| 320.0|            5976|                      32|              189|
|SHOP008|            2| 300.0|            5976|                      32|              189|
|SHOP112|            5|1050.0|            7997|                      36|             1308|
|SHOP055|            2| 300.0|            5604|                      37|             1370|
|SHOP112|            2| 440.0|            4654|                      36|             1308|
|SHOP112|            2| 540.0|            5904|                      36|             1308|

**Handle Quantity and Sales**

In [14]:
from pyspark.sql.functions import sum, col, avg, round

df_pyspark_1 = df_pyspark_1.withColumn("Sale", col("Sale").cast("float"))

New_alias_3 = df_pyspark_1.groupBy("shop_id").agg(sum("Sale").alias("Total_Sale"))
df_pyspark_1 = df_pyspark_1.join(New_alias_3, "shop_id", "left")
df_pyspark_1.show()

+-------+-------------+------+----------------+------------------------+-----------------+----------+
|shop_id|quantity_sold|  Sale|Sale_in_each_day|Distinct_item_count_sold|Customors_engaged|Total_Sale|
+-------+-------------+------+----------------+------------------------+-----------------+----------+
|SHOP008|            2| 440.0|           11713|                      32|              189|  131245.0|
|SHOP112|            2| 440.0|            7440|                      36|             1308| 1223905.0|
|SHOP008|            2| 320.0|            5976|                      32|              189|  131245.0|
|SHOP008|            2| 300.0|            5976|                      32|              189|  131245.0|
|SHOP112|            5|1050.0|            7997|                      36|             1308| 1223905.0|
|SHOP055|            2| 300.0|            5604|                      37|             1370|  931695.0|
|SHOP112|            2| 440.0|            4654|                      36|          

In [15]:
New_alias_4 = df_pyspark_1.groupBy("shop_id").agg(sum("quantity_sold").alias("Total_quantity_sold"))
df_pyspark_1 = df_pyspark_1.join(New_alias_4, "shop_id", "left")
df_pyspark_1.show()

+-------+-------------+------+----------------+------------------------+-----------------+----------+-------------------+
|shop_id|quantity_sold|  Sale|Sale_in_each_day|Distinct_item_count_sold|Customors_engaged|Total_Sale|Total_quantity_sold|
+-------+-------------+------+----------------+------------------------+-----------------+----------+-------------------+
|SHOP008|            2| 440.0|           11713|                      32|              189|  131245.0|              502.0|
|SHOP112|            2| 440.0|            7440|                      36|             1308| 1223905.0|             4609.0|
|SHOP008|            2| 320.0|            5976|                      32|              189|  131245.0|              502.0|
|SHOP008|            2| 300.0|            5976|                      32|              189|  131245.0|              502.0|
|SHOP112|            5|1050.0|            7997|                      36|             1308| 1223905.0|             4609.0|
|SHOP055|            2| 

In [16]:
New_alias_5 = df_pyspark_1.groupBy("shop_id").agg(round(avg("Sale_in_each_day")).alias("Average_daily_sale"))
df_pyspark_1 = df_pyspark_1.join(New_alias_5, "shop_id", "left")
df_pyspark_1.show()

+-------+-------------+------+----------------+------------------------+-----------------+----------+-------------------+------------------+
|shop_id|quantity_sold|  Sale|Sale_in_each_day|Distinct_item_count_sold|Customors_engaged|Total_Sale|Total_quantity_sold|Average_daily_sale|
+-------+-------------+------+----------------+------------------------+-----------------+----------+-------------------+------------------+
|SHOP008|            2| 440.0|           11713|                      32|              189|  131245.0|              502.0|            9658.0|
|SHOP112|            2| 440.0|            7440|                      36|             1308| 1223905.0|             4609.0|            6469.0|
|SHOP008|            2| 320.0|            5976|                      32|              189|  131245.0|              502.0|            9658.0|
|SHOP008|            2| 300.0|            5976|                      32|              189|  131245.0|              502.0|            9658.0|
|SHOP112|    

In [17]:
df_pyspark_1 = df_pyspark_1.drop('quantity_sold','Sale','Sale_in_each_day')

In [18]:
distinct_ids = df_pyspark_1.dropDuplicates()
distinct_ids.show()

+-------+------------------------+-----------------+----------+-------------------+------------------+
|shop_id|Distinct_item_count_sold|Customors_engaged|Total_Sale|Total_quantity_sold|Average_daily_sale|
+-------+------------------------+-----------------+----------+-------------------+------------------+
|SHOP012|                      33|             1263| 1169405.0|             4165.0|            6284.0|
|SHOP048|                      37|             1678| 1791290.0|             5624.0|            6380.0|
|SHOP047|                      36|              928|  842960.0|             3288.0|            6225.0|
|SHOP046|                      36|             1736| 2607865.0|             6941.0|            6227.0|
|SHOP049|                      36|             2075| 1970175.0|             7764.0|            6345.0|
|SHOP025|                      36|             1667| 1836340.0|             6333.0|            6607.0|
|SHOP078|                      36|             2073| 2663785.0|          

In [19]:
pandas_df = distinct_ids.toPandas()
pandas_df.head(6)

,shop_id,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
0,SHOP012,33,1263,1169405.0,4165.0,6284.0
1,SHOP048,37,1678,1791290.0,5624.0,6380.0
2,SHOP047,36,928,842960.0,3288.0,6225.0
3,SHOP046,36,1736,2607865.0,6941.0,6227.0
4,SHOP049,36,2075,1970175.0,7764.0,6345.0
5,SHOP025,36,1667,1836340.0,6333.0,6607.0


In [20]:
pandas_df.shape

(124, 6)

In [21]:
result = pandas_df[pandas_df['shop_id'] == 'SHOP046']
result

,shop_id,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
3,SHOP046,36,1736,2607865.0,6941.0,6227.0


In [22]:
store_df = pd.read_csv("/kaggle/input/data-storm-4/Store-info.csv")
store_df.head()

,shop_id,shop_area_sq_ft,shop_profile
0,SHOP047,528,Moderate
1,SHOP009,676,High
2,SHOP083,676,Low
3,SHOP117,676,Low
4,SHOP042,676,Low


In [23]:
# Join two tables
df = pd.merge(store_df, pandas_df, how='inner', left_on='shop_id', right_on='shop_id')
df.head(5)

,shop_id,shop_area_sq_ft,shop_profile,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
0,SHOP047,528,Moderate,36,928,842960.0,3288.0,6225.0
1,SHOP009,676,High,36,2498,1970870.0,8411.0,6404.0
2,SHOP083,676,Low,37,1900,1691985.0,6644.0,6387.0
3,SHOP117,676,Low,36,2037,2325980.0,8073.0,6356.0
4,SHOP042,676,Low,36,1841,1340215.0,5844.0,6402.0


In [24]:
df.shape

(124, 8)

In [25]:
result = df[df['shop_id'] == 'SHOP046']
result

,shop_id,shop_area_sq_ft,shop_profile,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
100,SHOP046,545,NaN,36,1736,2607865.0,6941.0,6227.0


In [26]:
# save for the test
nan_df = df.isna()
test_data = df.loc[nan_df.any(axis=1)]
test_data.tail(5)

,shop_id,shop_area_sq_ft,shop_profile,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
119,SHOP087,527,NaN,36,1083,1128935.0,3750.0,6385.0
120,SHOP050,411,NaN,32,1778,2399125.0,7668.0,6575.0
121,SHOP061,699,NaN,36,1746,2110800.0,6053.0,6490.0
122,SHOP056,597,NaN,36,2832,3023805.0,11163.0,6407.0
123,SHOP070,676,NaN,37,1423,1325070.0,5028.0,6390.0


In [27]:
train_df = df.dropna()
train_df.shape

(100, 8)

In [28]:
enchode_dict = {"Low":1 ,"Moderate" : 2, "High": 3}

In [29]:
train_df['shop_profile_enchoded'] = train_df['shop_profile'].map(enchode_dict)
train_df.head(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,shop_id,shop_area_sq_ft,shop_profile,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale,shop_profile_enchoded
0,SHOP047,528,Moderate,36,928,842960.0,3288.0,6225.0,2
1,SHOP009,676,High,36,2498,1970870.0,8411.0,6404.0,3
2,SHOP083,676,Low,37,1900,1691985.0,6644.0,6387.0,1
3,SHOP117,676,Low,36,2037,2325980.0,8073.0,6356.0,1
4,SHOP042,676,Low,36,1841,1340215.0,5844.0,6402.0,1


In [30]:
train_df = train_df.drop("shop_profile",axis=1)

In [31]:
train_df.sample(10)

,shop_id,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale,shop_profile_enchoded
14,SHOP091,676,36,2621,2465805.0,9389.0,6360.0,3
85,SHOP007,617,36,1787,2406355.0,7191.0,6323.0,1
6,SHOP100,617,36,2496,2946220.0,9369.0,6543.0,2
73,SHOP040,541,36,1603,1832065.0,6174.0,6395.0,2
48,SHOP055,720,37,1370,931695.0,4365.0,6239.0,2
20,SHOP051,676,36,2175,2441265.0,8570.0,6240.0,3
46,SHOP063,676,36,2181,2238925.0,7367.0,6456.0,3
43,SHOP103,706,36,1368,1707520.0,5102.0,6409.0,2
76,SHOP111,617,36,2959,3606120.0,11938.0,6250.0,3
75,SHOP082,549,36,3007,2694095.0,11083.0,6353.0,3


In [32]:
train_df.to_csv('Train.csv',index=False)

### **Handle Test Data**

In [33]:
testing_data_provided =  pd.read_csv('/kaggle/input/data-storm-4/Testing-data.csv')
testing_data_provided.head()

,shop_id,shop_profile
0,SHOP046,NaN
1,SHOP024,NaN
2,SHOP023,NaN
3,SHOP097,NaN
4,SHOP044,NaN


In [34]:
# Join two tables
test_df = pd.merge(testing_data_provided, test_data, how='inner', left_on='shop_id', right_on='shop_id')
test_df.head(5)

,shop_id,shop_profile_x,shop_area_sq_ft,shop_profile_y,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
0,SHOP046,NaN,545,NaN,36,1736,2607865.0,6941.0,6227.0
1,SHOP024,NaN,676,NaN,36,1641,1308795.0,5274.0,6176.0
2,SHOP023,NaN,617,NaN,37,2227,2570425.0,7946.0,6339.0
3,SHOP097,NaN,310,NaN,32,1669,2642380.0,6934.0,6355.0
4,SHOP044,NaN,715,NaN,36,1552,1310440.0,4965.0,6369.0


In [35]:
test_df.to_csv('Test.csv',index=False)

In [36]:
test_df = test_df.drop(["shop_profile_x","shop_profile_y","shop_id"],axis=1)
test_df.head(5)

,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
0,545,36,1736,2607865.0,6941.0,6227.0
1,676,36,1641,1308795.0,5274.0,6176.0
2,617,37,2227,2570425.0,7946.0,6339.0
3,310,32,1669,2642380.0,6934.0,6355.0
4,715,36,1552,1310440.0,4965.0,6369.0


## **SPLIT and TRAIN**

In [37]:
Y = train_df["shop_profile_enchoded"]
X = train_df.drop(["shop_profile_enchoded","shop_id"], axis=1)

In [38]:
Y.shape

(100,)

In [39]:
X.shape

(100, 6)

In [40]:
test_df.shape

(24, 6)

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=20)

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [43]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=10)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred)
f1_dt = f1_score(y_test, y_pred, average='micro')

# Print the accuracy of the classifier
print("Accuracy DT : ", accuracy_dt)
print("F1 score : ", f1_dt)

Accuracy DT :  0.6666666666666666
F1 score :  0.6666666666666666


In [44]:
from sklearn.ensemble import RandomForestClassifier
rf =  RandomForestClassifier()


rf.fit(X_train, y_train)
y_pred_rf = dt.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='micro')

# Print the accuracy of the classifier
print("Accuracy RF : ", accuracy_rf)
print("F1 score RF : ", f1_rf)

Accuracy RF :  0.6666666666666666
F1 score RF :  0.6666666666666666


In [45]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_pred_gnb = gnb.predict(X_test)

accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
f1_gnb = f1_score(y_test, y_pred_gnb, average='micro')

# Print the accuracy of the classifier
print("Accuracy GNB : ", accuracy_gnb)
print("F1 score GNB : ", f1_gnb)

Accuracy GNB :  0.8
F1 score GNB :  0.8000000000000002


In [46]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)

accuracy_svc = accuracy_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_rf, average='micro')

# Print the accuracy of the classifier
print("Accuracy SVC: ", accuracy_svc)
print("F1 score SVC : ", f1_svc)

Accuracy SVC:  0.4
F1 score SVC :  0.6666666666666666


In [47]:
# GNB IS BEST CLASSIFIER THIS FAR
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X, Y)

y_pred_gnb_f = gnb.predict(test_df)

In [48]:
print(y_pred_gnb_f)

[1 2 1 2 1 1 1 1 3 1 1 1 1 3 2 1 1 3 1 2 1 1 3 1]


In [49]:
submission = test_df
submission.head()

,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale
0,545,36,1736,2607865.0,6941.0,6227.0
1,676,36,1641,1308795.0,5274.0,6176.0
2,617,37,2227,2570425.0,7946.0,6339.0
3,310,32,1669,2642380.0,6934.0,6355.0
4,715,36,1552,1310440.0,4965.0,6369.0


In [50]:
submission['predicted'] = y_pred_gnb_f
submission.head()

,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale,predicted
0,545,36,1736,2607865.0,6941.0,6227.0,1
1,676,36,1641,1308795.0,5274.0,6176.0,2
2,617,37,2227,2570425.0,7946.0,6339.0,1
3,310,32,1669,2642380.0,6934.0,6355.0,2
4,715,36,1552,1310440.0,4965.0,6369.0,1


In [51]:
dechode_dict = {1 : "Low" , 2 : "Moderate" , 3 : "High" }

submission['shop_profile'] = submission['predicted'].map(dechode_dict)
submission.head(5)

,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale,predicted,shop_profile
0,545,36,1736,2607865.0,6941.0,6227.0,1,Low
1,676,36,1641,1308795.0,5274.0,6176.0,2,Moderate
2,617,37,2227,2570425.0,7946.0,6339.0,1,Low
3,310,32,1669,2642380.0,6934.0,6355.0,2,Moderate
4,715,36,1552,1310440.0,4965.0,6369.0,1,Low


In [52]:
submission_1 = pd.merge(testing_data_provided, submission, how='inner', left_index=True, right_index=True)
submission_1.head(5)

,shop_id,shop_profile_x,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale,predicted,shop_profile_y
0,SHOP046,NaN,545,36,1736,2607865.0,6941.0,6227.0,1,Low
1,SHOP024,NaN,676,36,1641,1308795.0,5274.0,6176.0,2,Moderate
2,SHOP023,NaN,617,37,2227,2570425.0,7946.0,6339.0,1,Low
3,SHOP097,NaN,310,32,1669,2642380.0,6934.0,6355.0,2,Moderate
4,SHOP044,NaN,715,36,1552,1310440.0,4965.0,6369.0,1,Low


In [53]:
submission_1["shop_profile"]  = submission_1["shop_profile_y"]
submission_1.head(5)

,shop_id,shop_profile_x,shop_area_sq_ft,Distinct_item_count_sold,Customors_engaged,Total_Sale,Total_quantity_sold,Average_daily_sale,predicted,shop_profile_y,shop_profile
0,SHOP046,NaN,545,36,1736,2607865.0,6941.0,6227.0,1,Low,Low
1,SHOP024,NaN,676,36,1641,1308795.0,5274.0,6176.0,2,Moderate,Moderate
2,SHOP023,NaN,617,37,2227,2570425.0,7946.0,6339.0,1,Low,Low
3,SHOP097,NaN,310,32,1669,2642380.0,6934.0,6355.0,2,Moderate,Moderate
4,SHOP044,NaN,715,36,1552,1310440.0,4965.0,6369.0,1,Low,Low


In [54]:
sumbission_last = submission_1.loc[:, ['shop_id', 'shop_profile']]

In [55]:
sumbission_last.to_csv('submission.csv',index=False)